In [1]:
# -*- coding: utf-8 -*-

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

tf.reset_default_graph()

start_time = time.time()

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

learning_rate = 0.01
training_epochs = 20
batch_size = 100
display_step = 1

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

dropout_rate = tf.placeholder("float")

W1 = tf.get_variable(name="W1", shape=[3,3,1,32], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(name="W2", shape=[3,3,32,64], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(name="W3", shape=[3,3,64,128], initializer=tf.contrib.layers.xavier_initializer())
W4 = tf.get_variable(name="W4", shape=[3,3,128,256], initializer=tf.contrib.layers.xavier_initializer())
W5 = tf.get_variable(name="W5", shape=[128, 10], initializer=tf.contrib.layers.xavier_initializer())

L = tf.reshape(X, [-1, 28, 28, 1])

L1_c = tf.nn.conv2d(L, W1, strides=[1,1,1,1], padding='SAME') # ?, 28, 28, 32
L1_r = tf.nn.relu(L1_c)
L1_d = tf.nn.max_pool(L1_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 14, 14, 32
L1 = tf.nn.dropout(L1_d, dropout_rate)

L2_c = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME') # ?, 14, 14, 64
L2_r = tf.nn.relu(L2_c)
L2_d = tf.nn.max_pool(L2_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 7, 7, 64
L2 = tf.nn.dropout(L2_d, dropout_rate)

L3_c = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME') # ?, 7, 7, 128
L3_r = tf.nn.relu(L3_c)
L3_d = tf.nn.max_pool(L3_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 4, 4, 128
L3 = tf.nn.dropout(L3_d, dropout_rate)

col_w = tf.get_variable(name="col_sum", shape=[4, 128], initializer=tf.ones_initializer())
row_w = tf.get_variable(name="row_sum", shape=[4, 128], initializer=tf.ones_initializer())

col_sum = tf.reduce_sum(L3, axis=1)# * col_w
row_sum = tf.reduce_sum(col_sum, axis=1)# * row_w
#L3_sum = tf.concat([col_sum, row_sum], axis=1)

L3_relu = tf.nn.relu(row_sum)
L4_d = tf.reshape(L3_relu, [-1, 128])

L5_result = tf.matmul(L4_d, W5)
L5_d = tf.nn.dropout(L5_result, dropout_rate)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=L5_d, labels=Y))
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

init = tf.global_variables_initializer()

"""session start"""
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys, dropout_rate:1.0})
            
            avg_cost += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        #print(col_w.eval())
        #print(row_w.eval())
    correct_prediction = tf.equal(tf.argmax(L5_d, 1), tf.argmax(Y, 1))
        
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, dropout_rate: 1.0}))
    sess.close()
    
end_time = time.time()

print("time: ", end_time-start_time)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 0.765769024
Epoch: 0002 cost= 0.096966286
Epoch: 0003 cost= 0.056949937
Epoch: 0004 cost= 0.040100810
Epoch: 0005 cost= 0.032399656
Epoch: 0006 cost= 0.025306679


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 29 20:48:23 2018

@author: jsc5565
"""


import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time
tf.reset_default_graph()

mnist = input_data.read_data_sets("mnist", one_hot=True)

learning_rate = 0.001
train_epochs = 15
train_size = 100
display_step = 1
filter_size = 20
img_size = 14

x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.float32, [None, 10])

row_w = tf.get_variable(name="row_w", shape=[img_size, filter_size], initializer=tf.ones_initializer())
col_w = tf.get_variable(name="col_w", shape=[img_size, filter_size], initializer=tf.ones_initializer())
filter_w = tf.Variable(tf.random_normal([7,7,1,filter_size], stddev=0.01))
output = tf.get_variable(name="output", shape=[img_size*filter_size, 10], initializer=tf.ones_initializer())
logit = tf.get_variable(name="logit", shape=[img_size*img_size, 10], initializer=tf.ones_initializer())

w = tf.get_variable(name="w", shape=[14*14*filter_size, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name="b", shape=[10], initializer=tf.contrib.layers.xavier_initializer())

x_re = tf.reshape(x, [-1, 28, 28, 1])
x_conv = tf.nn.conv2d(x_re, filter_w, strides=[1,1,1,1], padding='SAME')
x_pool = tf.nn.max_pool(x_conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #?, 14, 14, filter_size

"""first conv2d"""
"""
mul1 = x_pool #* row_w
transpose = tf.transpose(mul1, perm=[0, 2, 1, 3])
mul2 = transpose# * col_w
"""

col_sum = tf.reduce_sum(x_pool, axis=1) * row_w
row_sum = tf.reduce_sum(x_pool, axis=2) * col_w

col_sum_re = tf.reshape(col_sum, [-1, img_size*filter_size])
row_sum_re = tf.reshape(row_sum, [-1, img_size*filter_size])

total_sum = col_sum_re + row_sum_re
relu_add = tf.nn.relu(total_sum)

#esult = tf.matmul(relu_add, output)

result = tf.nn.relu(tf.matmul(tf.reshape(x_pool,[-1, 14*14*filter_size]), w) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


"""sesson start"""

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    start_time = time.time()
    for epoch in range(train_epochs):
        avg_cost = 0
        
        total_epochs = int(mnist.train.num_examples/ train_size)
        
        for i in range(total_epochs):
            batch_xs, batch_ys = mnist.train.next_batch(train_size)
            _,c, row= sess.run([optimizer, cost, row_w], {x: batch_xs, y: batch_ys})
            avg_cost += c/total_epochs
            
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:9f}".format(avg_cost))
            
    prediction = tf.equal(tf.argmax(result, 1), tf.argmax(y, 1))            
            
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y:mnist.test.labels}))
    
    sess.close()
    
    end_time = time.time()
    
    print('time:', end_time-start_time)

In [ ]:
# -*- coding: utf-8 -*-

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

tf.reset_default_graph()

start_time = time.time()

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

learning_rate = 0.01
training_epochs = 20
batch_size = 100
display_step = 1

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

dropout_rate = tf.placeholder("float")

W1 = tf.get_variable(name="W1", shape=[3,3,1,32], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(name="W2", shape=[3,3,32,64], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(name="W3", shape=[3,3,64,128], initializer=tf.contrib.layers.xavier_initializer())
W4 = tf.get_variable(name="W4", shape=[3,3,128,256], initializer=tf.contrib.layers.xavier_initializer())
W5 = tf.get_variable(name="W5", shape=[128, 10], initializer=tf.contrib.layers.xavier_initializer())

L = tf.reshape(X, [-1, 28, 28, 1])

L1_c = tf.nn.conv2d(L, W1, strides=[1,1,1,1], padding='SAME') # ?, 28, 28, 32
L1_r = tf.nn.relu(L1_c)
L1_d = tf.nn.max_pool(L1_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 14, 14, 32
L1 = tf.nn.dropout(L1_d, dropout_rate)

L2_c = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME') # ?, 14, 14, 64
L2_r = tf.nn.relu(L2_c)
L2_d = tf.nn.max_pool(L2_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 7, 7, 64
L2 = tf.nn.dropout(L2_d, dropout_rate)

L3_c = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME') # ?, 7, 7, 128
L3_r = tf.nn.relu(L3_c)
L3_d = tf.nn.max_pool(L3_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ?, 4, 4, 128
L3_dr = tf.nn.max_pool(L3_d, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3_dr2 = tf.nn.max_pool(L3_dr, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3_sq = tf.squeeze(L3_dr2)
#L3 = tf.nn.dropout(L3_d, dropout_rate)

#L4_d = tf.reshape(L3, [-1, 128])

L5_result = tf.matmul(L3_sq, W5)
L5_d = tf.nn.dropout(L5_result, dropout_rate)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=L5_d, labels=Y))
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

init = tf.global_variables_initializer()

"""session start"""
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys, dropout_rate:1.0})
            
            avg_cost += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        #print(col_w.eval())
        #print(row_w.eval())
    correct_prediction = tf.equal(tf.argmax(L5_d, 1), tf.argmax(Y, 1))
        
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, dropout_rate: 1.0}))
    sess.close()
    
end_time = time.time()

print("time: ", end_time-start_time)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time
tf.reset_default_graph()

mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

batch_size = 100
training_epochs = 30
learning_rate = 0.001
display_step = 1

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

w = tf.get_variable(name="w", shape=[784, 1024], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name="b", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())

output_w = tf.get_variable(name="output_w", shape=[1024, 10], initializer=tf.contrib.layers.xavier_initializer())

def Activation(matrix):
    output_save = 1.1/(1+tf.exp(-3*matrix))-0.05
    #output_save = 1/tf.tanh(matrix)
    output_max = tf.minimum(1.0, output_save)
    output_min = tf.maximum(-1.0, output_max)
    return output_save


L1 = Activation(tf.matmul(x, w)+b)
#L1 = tf.nn.relu(tf.matmul(x,w)+b)
result = tf.matmul(L1, output_w)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init)
    start_time = time.time()
    for epoch in range(training_epochs):
        avg_cost = 0
        
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {x:batch_xs, y:batch_ys})
            
            avg_cost += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            
        if avg_cost < 0.01:
            learning_rate = 0.0001
        
    correct_prediction = tf.equal(tf.argmax(result, 1), tf.argmax(y, 1))
        
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    sess.close()
    
    end_time = time.time()
    
    print("time:", end_time - start_time)

In [ ]:
# -*- coding: utf-8 -*-

#learning_rate = 0.01, cost= 7.652248363, Accuracy: 0.9675

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

tf.reset_default_graph()

learning_rate = 0.05
training_epochs = 15
batch_size = 100
display_step = 1

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

dropout_rate = tf.placeholder("float")

W1 = tf.get_variable(name="w1", shape=[784, 1024], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(name="w2", shape=[1024, 1024], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(name="w3", shape=[1024, 1024], initializer=tf.contrib.layers.xavier_initializer())
W4 = tf.get_variable(name="w4", shape=[1024, 1024], initializer=tf.contrib.layers.xavier_initializer())
W5 = tf.get_variable(name="w5", shape=[784*784, 10], initializer=tf.contrib.layers.xavier_initializer())

B1 = tf.get_variable(name="b1", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.get_variable(name="b2", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())
B3 = tf.get_variable(name="b3", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())
B4 = tf.get_variable(name="b4", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())
B5 = tf.get_variable(name="b5", shape=[10], initializer=tf.contrib.layers.xavier_initializer())

def Activation(matrix):
    #output_save = 1.1/(1+tf.exp(-3*matrix))-0.05
    #output_save = 2/(1+tf.exp(-3*matrix))
    output_save = matrix
    #output_max = tf.minimum(1.0, output_save)
    output_min = tf.maximum(0.0, output_save)
    return output_min
"""
L1 = Activation(tf.add(tf.matmul(X, W1), B1))
L2 = Activation(tf.add(tf.matmul(L1, W2), B2))
L3 = Activation(tf.add(tf.matmul(L2, W3), B3))
L4 = Activation(tf.add(tf.matmul(L3, W4), B4))
result = tf.add(tf.matmul(L4, W5), B5)
"""
prev = tf.reshape(tf.matmul(tf.reshape(X, [-1, 784, 1]), tf.reshape(X, [-1, 1, 784])), [-1, 784*784])
#result = tf.add(tf.matmul(tf.matmul(X, tf.reshape(X,[-1, 784])),W5), B5)
result = tf.add(tf.matmul(prev, W5), B5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init)
    start_time = time.time()
    for epoch in range(training_epochs):
        avg_cost = 0
        
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys})
            
            avg_cost += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        
    correct_prediction = tf.equal(tf.argmax(result, 1), tf.argmax(Y, 1))
        
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    sess.close()
    
    end_time = time.time()
    
    print("time:", end_time - start_time)

In [ ]:
# -*- coding: utf-8 -*-

#learning_rate = 0.01, cost= 7.652248363, Accuracy: 0.9675

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

tf.reset_default_graph()

learning_rate = 0.01
training_epochs = 15
batch_size = 100
display_step = 1

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

dropout_rate = tf.placeholder("float")

W1 = tf.get_variable(name="w1", shape=[784, 1024], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(name="w2", shape=[1024, 10], initializer=tf.contrib.layers.xavier_initializer())

B1 = tf.get_variable(name="b1", shape=[1024], initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.get_variable(name="b2", shape=[10], initializer=tf.contrib.layers.xavier_initializer())

L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))
result = tf.add(tf.matmul(L1, W2), B2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)
    
    start_time = time.time()
    for epoch in range(training_epochs):
        avg_cost = 0
        
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys})
            
            avg_cost += c/total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        
    correct_prediction = tf.equal(tf.argmax(result, 1), tf.argmax(Y, 1))
        
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    sess.close()
    
    end_time = time.time()
    
    print("time:", end_time - start_time)